In [7]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import os
from PIL import Image
from io import BytesIO
import pytesseract
import base64


In [8]:

URL = 'https://www.tratencongty.com/'  # Đổi URL này thành URL chính của bạn
START_PAGE = 1  # Thay đổi số trang bắt đầu nếu cần
BASE_URL = 'https://www.tratencongty.com/'

FIELDS = [
    'Tên công ty',
    'Loại hình hoạt động',
    'Mã số thuế',
    'Địa chỉ',
    'Đại diện pháp luật',
    'Ngày cấp giấy phép',
    'Ngày hoạt động',
    'Điện thoại trụ sở',
    'Trạng thái'
]


In [9]:

data = []

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


In [10]:

def init_data():
    file_path = get_file_path()
    if not os.path.exists(file_path):
        return []
    
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    
    headers = [cell.value for cell in sheet[1]]
    
    data = []
    for row in sheet.iter_rows(min_row=2, values_only=True):
        row_dict = dict(zip(headers, row))
        data.append(row_dict)
    
    return data

def get_file_path():
    return f"./data3.xlsx"

def write_file():
    file_path = get_file_path()
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(FIELDS)
    for company in data:
        row = [company.get(field, '') for field in FIELDS]
        sheet.append(row)
    workbook.save(file_path)

def company_exists(new_company):
    for company in data:
        if company['Tên công ty'] == new_company['Tên công ty'] and company['Mã số thuế'] == new_company['Mã số thuế']:
            return True
    return False

def extract_text_from_base64_image(base64_string):
    base64_data = base64_string.split(',')[1]
    img_data = base64.b64decode(base64_data)
    img = Image.open(BytesIO(img_data))
    text = pytesseract.image_to_string(img, config='--psm 6')
    return text.strip()

def get_company_data(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    company = {}
    jumbotron = soup.select_one('.jumbotron')
    
    if jumbotron:
        name = jumbotron.select_one('h4 a').text.strip()
        company['Tên công ty'] = name
        
        info_br = str(jumbotron).split('<br/>')
        for line in info_br:
            if 'Loại hình hoạt động' in line:
                company['Loại hình hoạt động'] = line.split('Loại hình hoạt động:', 1)[1].strip()
            elif 'Mã số thuế' in line:
                img_base64 = BeautifulSoup(line, 'html.parser').select_one('img')['src']
                company['Mã số thuế'] = extract_text_from_base64_image(img_base64)
            elif 'Địa chỉ' in line:
                company['Địa chỉ'] = line.split(':', 1)[1].strip()
            elif 'Đại diện pháp luật' in line:
                company['Đại diện pháp luật'] = line.split(':', 1)[1].strip()
            elif 'Ngày cấp giấy phép' in line:
                company['Ngày cấp giấy phép'] = line.split(':', 1)[1].strip()
            elif 'Ngày hoạt động' in line:
                company['Ngày hoạt động'] = BeautifulSoup(line, 'html.parser').text.split(':', 1)[1].strip()
            elif 'Điện thoại trụ sở' in line:
                img_base64 = BeautifulSoup(line, 'html.parser').select_one('img')['src']
                company['Điện thoại trụ sở'] = extract_text_from_base64_image(img_base64)
            elif 'Trạng thái' in line:
                company['Trạng thái'] = line.split(':', 1)[1].strip()
        company['url'] = url
        
        if not company_exists(company):
            data.append(company)
    else:
        print(f"No jumbotron found for URL: {url}")


In [11]:
def get_company_links(page):
    response = requests.get(f"{URL}/page-{page}", headers=HEADERS)
    print(f"Fetching links from page {page}, Status code: {response.status_code}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    links = [a['href'] for a in soup.select('.search-results a')]
    print(f"Links found: {links}")
    return links

def main():
    global data
    data = init_data()
    for page in range(START_PAGE, 2):  # Điều chỉnh số lượng trang nếu cần
        print(f"Processing page {page}")
        links = get_company_links(page)
        for link in links:
            get_company_data(link)
        write_file()

if __name__ == "__main__":
    main()

Processing page 1
Fetching links from page 1, Status code: 200
Links found: ['https://www.tratencongty.com/company/3fbdd174-cong-ty-luat-trach-nhiem-huu-han-kvn/', 'https://www.tratencongty.com/company/3fbdd174-cong-ty-luat-trach-nhiem-huu-han-kvn/', 'https://www.tratencongty.com/company/3fbdd16d-van-phong-dai-dien-fuji-print-industrial-h-k-company-limited-tai-thanh-pho-ha-noi/', 'https://www.tratencongty.com/company/3fbdd16d-van-phong-dai-dien-fuji-print-industrial-h-k-company-limited-tai-thanh-pho-ha-noi/', 'https://www.tratencongty.com/company/3fbdd166-ban-quan-ly-du-an-ho-tro-ky-thuat-mlc/', 'https://www.tratencongty.com/company/3fbdd166-ban-quan-ly-du-an-ho-tro-ky-thuat-mlc/', 'https://www.tratencongty.com/company/453132d0-cong-ty-tnhh-thuong-mai-va-dich-vu-logistics-an-thuan-phat/', 'https://www.tratencongty.com/company/453132d0-cong-ty-tnhh-thuong-mai-va-dich-vu-logistics-an-thuan-phat/', 'https://www.tratencongty.com/company/bc50cc42-cong-ty-tnhh-mot-thanh-vien-duyen-dang/', 'h